In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

%reload_ext dotenv

In [7]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# LOGIN

In [8]:
url = '''https://www.linkedin.com/company/soujunior/posts/?feedView=all'''
driver.get(url)

In [9]:
xpath_login = "/html/body/div/main/div[2]/div[1]/form/div[1]/input"
xpath_password = "/html/body/div/main/div[2]/div[1]/form/div[2]/input"
xpath_submit = "/html/body/div/main/div[2]/div[1]/form/div[3]/button"

login_linkedin = os.getenv("LOGIN_LINKEDIN")
password_linkedin = os.getenv("PASSWORD_LINKEDIN")

In [10]:
input_login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_login)))
input_password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_password)))
input_submit = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_submit)))

In [11]:
input_login.clear()
input_password.clear()

input_login.send_keys(login_linkedin)
input_password.send_keys(password_linkedin)
input_submit.click()

In [22]:
# sort by date
class_sort = "sort-dropdown__dropdown"

sort_div = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, class_sort))
)

sort_button = sort_div.find_element(By.TAG_NAME, "button")
sort_button.click()

sort_by_date = sort_div.find_elements(By.TAG_NAME, "li")[1]
sort_by_date.click()

# SCRAP

In [23]:
body = driver.find_element(By.TAG_NAME, "body")

for _ in range(70):
    body.send_keys(Keys.PAGE_DOWN)

    time.sleep(2)

In [27]:
class_feed_posts = "scaffold-finite-scroll__content"

feed_posts = driver.find_element(By.CLASS_NAME, class_feed_posts)

posts = feed_posts.find_elements(By.XPATH, './div')


In [37]:
# save posts to html
output_folder = input("Output: ")
os.mkdir(f"linkedin_page/scraps/{output_folder}")
os.mkdir(f"linkedin_page/scraps/{output_folder}/profile_pics")

screenshoted_author_list = []
for index, post in enumerate(posts):

    author = post.find_elements(By.CLASS_NAME, "update-components-actor__name")

    # validation
    if author:
        author_name = author[0].find_element(By.CLASS_NAME, "visually-hidden").text
    else:
        continue
    
    if author_name not in screenshoted_author_list:
        img = post.find_element(By.CLASS_NAME, 'update-components-actor__avatar-image')
        img_path = f"linkedin_page/scraps/{output_folder}/profile_pics/{author_name}.png"
        img.screenshot(f"linkedin_page/scraps/{output_folder}/profile_pics/{author_name}.png")

        screenshoted_author_list.append(author_name)

    # save post
    with open(f'linkedin_page/scraps/{output_folder}/post_{index}.html', 'w', encoding='utf-8') as file:
        file.write(f'<p>{author_name}</p>\n')
        file.write(post.get_attribute('outerHTML'))